# 신용카드 크롤링 데이터 저장

- api 크롤링을 통해 카드고릴라 사이트의 모든 신용카드정보를 가져옵니다


## 1. 데이터 구조화 전략: "혜택 단위 분할"

일반적으로 카드 데이터는 '카드 1개' 단위로 저장되지만, 본 코드에서는 '카드 내 개별 혜택' 단위로 데이터를 생성합니다.

문제점: 카드 한 장에 담긴 모든 혜택(교통, 편의점, 배달 등)을 하나의 문서로 저장하면 텍스트가 너무 길어져 임베딩(Embedding) 시 의미가 희석됩니다.

해결책: get_rag_merged_data 함수를 통해 [카드명] + [카테고리] + [상세 내용] 조합의 개별 문서를 생성합니다.

예: "[신한카드 Mr.Life] 편의점 혜택: 세븐일레븐, CU 10% 할인"

In [ ]:
import requests
import json
import time
from bs4 import BeautifulSoup

# 최신 브라우저 헤더 설정
headers = {
    "authority": "api.card-gorilla.com:8080",
    "accept": "application/json, text/plain, */*",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36",
    "referer": "https://www.card-gorilla.com/"
}

def get_rag_merged_data(card_data):
    """동일 카드 내 중복 카테고리를 병합하여 RAG용 리스트 반환"""
    card_idx = card_data.get("idx")
    card_name = card_data.get("name")
    corp_name = card_data.get("corp_txt")
    
    detail_url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}"
    try:
        res = requests.get(detail_url, headers=headers, timeout=10)
        res_data = res.json()
        
        key_benefits = res_data.get('key_benefit', [])
        merged_benefits = {}
        
        for kb in key_benefits:
            category = kb.get('title')
            html_content = kb.get('info', '')
            
            if html_content:
                soup = BeautifulSoup(html_content, 'html.parser')
                clean_text = soup.get_text(separator=" ", strip=True)
                
                if category not in merged_benefits:
                    merged_benefits[category] = []
                
                if clean_text not in merged_benefits[category]:
                    merged_benefits[category].append(clean_text)
        
        results = []
        for category, texts in merged_benefits.items():
            combined_text = " / ".join(texts)
            full_content = f"[{card_name}] {category} 혜택: {combined_text}"
            
            results.append({
                "content": full_content, # 벡터 DB 임베딩 및 검색용
                "metadata": {            # 필터링용 메타데이터
                    "card_id": card_idx,
                    "card_name": card_name,
                    "corp": corp_name,
                    "category": category,
                    "min_performance": card_data.get("pre_month_money"),
                    "annual_fee": card_data.get("annual_fee_basic")
                }
            })
        return results
    except Exception as e:
        print(f"   ⚠️ Error (ID {card_idx}): {e}")
        return []

# --- 전체 카드 전수 수집 시작 ---
all_rag_data = []
all_card_indices = []
page = 1

print("🚀 모든 카드의 데이터 수집을 시작합니다. (전체 페이지 순회)")

while True:
    list_url = f"https://api.card-gorilla.com:8080/v1/cards?p={page}&perPage=20&cate=CRD"
    try:
        response = requests.get(list_url, headers=headers, timeout=10)
        data = response.json()
        goods_list = data.get("data", [])

        # 더 이상 수집할 카드가 없으면 종료
        if not goods_list:
            print(f"\n✅ 수집을 완료했습니다. (마지막 페이지: {page-1})")
            break

        print(f"\n📄 현재 {page}페이지 수집 중... (현재 누적 데이터: {len(all_rag_data)}개)")

        for card in goods_list:
            c_name = card.get('name')
            c_idx = card.get('idx')
            
            print(f"  📦 {c_name} (ID: {c_idx}) 처리 중...")
            
            # 1. idx 리스트 추가
            all_card_indices.append(c_idx)
            
            # 2. 혜택 병합 및 수집
            benefit_units = get_rag_merged_data(card)
            all_rag_data.extend(benefit_units)
            
            # 서버 부하 방지용 짧은 휴식
            time.sleep(0.4)

        page += 1 # 다음 페이지로

    except Exception as e:
        print(f"❌ {page}페이지 호출 중 오류 발생: {e}")
        break

# --- 최종 결과 저장 ---

# 1. RAG용 상세 데이터 저장 (JSON)
with open("all_card_benefits_rag.json", "w", encoding="utf-8") as f:
    json.dump(all_rag_data, f, ensure_ascii=False, indent=4)

# 2. 마스터 idx 리스트 저장 (JSON)
with open("all_card_idx_list.json", "w", encoding="utf-8") as f:
    json.dump(all_card_indices, f, ensure_ascii=False, indent=4)

print("\n" + "="*50)
print(f"✨ 최종 수집 완료!")
print(f"📂 상세 혜택 데이터: {len(all_rag_data)}개 ('all_credit_benefits_rag.json')")
print(f"📂 수집된 카드 ID: {len(all_card_indices)}개 ('all_credit_idx_list.json')")
print("="*50)

🚀 모든 카드의 데이터 수집을 시작합니다. (전체 페이지 순회)

📄 현재 1페이지 수집 중... (현재 누적 데이터: 0개)
  📦 신한카드 Mr.Life (ID: 13) 처리 중...
  📦 삼성 iD SELECT ALL 카드 (ID: 2885) 처리 중...
  📦 카드의정석 SHOPPING+ (ID: 2687) 처리 중...
  📦 삼성카드 & MILEAGE PLATINUM (스카이패스) (ID: 49) 처리 중...
  📦 KB국민 My WE:SH 카드 (ID: 2441) 처리 중...
  📦 삼성카드 taptap O (ID: 51) 처리 중...
  📦 쿠팡 와우카드 (ID: 2609) 처리 중...
  📦 American Express® Gold Card Edition2 (ID: 2663) 처리 중...
  📦 삼성 iD SELECT ON 카드 (ID: 2886) 처리 중...
  📦 삼성 iD SIMPLE 카드 (ID: 2376) 처리 중...
  📦 신한카드 Discount Plan+ (ID: 2835) 처리 중...
  📦 신한카드 처음(ANNIVERSE) (ID: 2759) 처리 중...
  📦 LOCA 365 카드 (ID: 2330) 처리 중...
  📦 taptap DIGITAL (ID: 657) 처리 중...
  📦 LOCA LIKIT 1.2 (ID: 2261) 처리 중...
  📦 올바른 FLEX 카드 (ID: 666) 처리 중...
  📦 카드의정석 EVERY DISCOUNT (ID: 2719) 처리 중...
  📦 카드의정석2 (ID: 2848) 처리 중...
  📦 신한카드 Deep Oil (ID: 39) 처리 중...
  📦 THE iD. 1st (ID: 2915) 처리 중...

📄 현재 2페이지 수집 중... (현재 누적 데이터: 121개)
  📦 삼성 iD GLOBAL 카드 (ID: 2676) 처리 중...
  📦 THE 1 (스카이패스) (ID: 1909) 처리 중...
  📦 우리카드 MILE&POINT (ID: 28

# 체크카드 크롤링 저장

- api 크롤링을 통해 카드고릴라 사이트의 모든 신용카드정보를 가져옵니다


In [27]:
import requests
import json
import time
from bs4 import BeautifulSoup

# 최신 브라우저 헤더 설정
headers = {
    "authority": "api.card-gorilla.com:8080",
    "accept": "application/json, text/plain, */*",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36",
    "referer": "https://www.card-gorilla.com/"
}

def get_rag_merged_data(card_data):
    """동일 카드 내 중복 카테고리를 병합하여 RAG용 리스트 반환"""
    card_idx = card_data.get("idx")
    card_name = card_data.get("name")
    corp_name = card_data.get("corp_txt")
    
    # 체크카드의 상세 정보 API 주소 (동일함)
    detail_url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}"
    try:
        res = requests.get(detail_url, headers=headers, timeout=10)
        res_data = res.json()
        
        key_benefits = res_data.get('key_benefit', [])
        merged_benefits = {}
        
        for kb in key_benefits:
            category = kb.get('title')
            html_content = kb.get('info', '')
            
            if html_content:
                soup = BeautifulSoup(html_content, 'html.parser')
                clean_text = soup.get_text(separator=" ", strip=True)
                
                if category not in merged_benefits:
                    merged_benefits[category] = []
                
                if clean_text not in merged_benefits[category]:
                    merged_benefits[category].append(clean_text)
        
        results = []
        for category, texts in merged_benefits.items():
            combined_text = " / ".join(texts)
            full_content = f"[{card_name}] {category} 혜택: {combined_text}"
            
            results.append({
                "content": full_content,
                "metadata": {
                    "card_id": card_idx,
                    "card_name": card_name,
                    "corp": corp_name,
                    "category": category,
                    "min_performance": card_data.get("pre_month_money"),
                    "annual_fee": card_data.get("annual_fee_basic"),
                    "card_type": "check" # 체크카드임을 명시
                }
            })
        return results
    except Exception as e:
        print(f"   ⚠️ Error (ID {card_idx}): {e}")
        return []

# --- 체크카드 전수 수집 시작 ---
all_check_data = []
all_check_indices = []
page = 1

print("🚀 모든 체크카드의 데이터 수집을 시작합니다. (cate=CHK)")

while True:
    # 카테고리를 CHK로 변경
    list_url = f"https://api.card-gorilla.com:8080/v1/cards?p={page}&perPage=20&cate=CHK"
    try:
        response = requests.get(list_url, headers=headers, timeout=10)
        data = response.json()
        goods_list = data.get("data", [])

        if not goods_list:
            print(f"\n✅ 수집 완료! (마지막 페이지: {page-1})")
            break

        print(f"\n📄 체크카드 {page}페이지 수집 중... (현재 누적: {len(all_check_data)}개)")

        for card in goods_list:
            c_name = card.get('name')
            c_idx = card.get('idx')
            
            print(f"  📦 {c_name} (ID: {c_idx}) 처리 중...")
            
            all_check_indices.append(c_idx)
            benefit_units = get_rag_merged_data(card)
            all_check_data.extend(benefit_units)
            
            time.sleep(0.4) # 서버 부하 방지

        page += 1

    except Exception as e:
        print(f"❌ {page}페이지 호출 중 오류 발생: {e}")
        break

# --- 최종 결과 저장 (파일명 구분) ---

with open("all_check_benefits_rag.json", "w", encoding="utf-8") as f:
    json.dump(all_check_data, f, ensure_ascii=False, indent=4)

with open("all_check_idx_list.json", "w", encoding="utf-8") as f:
    json.dump(all_check_indices, f, ensure_ascii=False, indent=4)

print("\n" + "="*50)
print(f"✨ 체크카드 수집 완료!")
print(f"📂 상세 혜택 데이터: {len(all_check_data)}개 ('all_check_benefits_rag.json')")
print(f"📂 수집된 카드 ID: {len(all_check_indices)}개 ('all_check_idx_list.json')")
print("="*50)

🚀 모든 체크카드의 데이터 수집을 시작합니다. (cate=CHK)

📄 체크카드 1페이지 수집 중... (현재 누적: 0개)
  📦 ONE 체크카드 (ID: 2749) 처리 중...
  📦 KB Youth Club 체크카드 (ID: 2929) 처리 중...
  📦 신한카드 SOL트래블 체크 (ID: 2667) 처리 중...
  📦 노리2 체크카드(KB Pay) (ID: 2422) 처리 중...
  📦 네이버페이 머니카드 (ID: 2626) 처리 중...
  📦 하나 나라사랑카드 (ID: 2934) 처리 중...
  📦 토스뱅크 체크카드 (ID: 2269) 처리 중...
  📦 트래블러스 체크카드(토심이) (ID: 2691) 처리 중...
  📦 KB 틴업 체크카드 (ID: 2852) 처리 중...
  📦 더나은 체크카드 (ID: 2601) 처리 중...
  📦 네이버페이 머니 하나 체크카드 (ID: 2432) 처리 중...
  📦 개이득 체크카드 (ID: 2519) 처리 중...
  📦 토심이 첵첵 체크카드 (ID: 2604) 처리 중...
  📦 신한카드 나라사랑카드 체크 (ID: 2933) 처리 중...
  📦 신한카드 Deep Dream 체크 (ID: 281) 처리 중...
  📦 카카오뱅크 프렌즈 체크카드 (ID: 435) 처리 중...
  📦 노리2 체크카드(Global) (ID: 2423) 처리 중...
  📦 카드의정석 오하CHECK (ID: 2570) 처리 중...
  📦 모빌리언스카드 (ID: 2321) 처리 중...
  📦 트래블로그 체크카드 (ID: 2394) 처리 중...

📄 체크카드 2페이지 수집 중... (현재 누적: 130개)
  📦 K-패스카드(체크) (ID: 2561) 처리 중...
  📦 노리체크카드 (ID: 348) 처리 중...
  📦 직장인보너스체크카드 (ID: 350) 처리 중...
  📦 트래블러스 체크카드 (ID: 2905) 처리 중...
  📦 NH20해봄체크카드 (ID: 362) 처리 중...
  📦 신한카드 O

In [28]:
import json
import pandas as pd

def check_json_quality(file_path):
    print(f"--- [{file_path}] 데이터 품질 체크 시작 ---")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 1. 데이터가 비어있는지 확인
    if not data:
        print("❌ 오류: 파일이 비어 있습니다.")
        return

    # Pandas 데이터프레임으로 변환 (분석 용이)
    df = pd.json_normalize(data)
    
    # 2. 필수 필드 누락 체크
    # content(혜택 내용), metadata.card_name, metadata.category 등
    required_columns = ['content', 'metadata.card_id', 'metadata.card_name', 'metadata.category']
    
    print(f"📊 총 데이터 개수: {len(df)}개")
    
    for col in required_columns:
        if col in df.columns:
            missing_count = df[col].isna().sum() or (df[col] == "").sum()
            if missing_count > 0:
                print(f"⚠️ 경고: {col} 필드에 {missing_count}개의 누락 데이터가 있습니다.")
            else:
                print(f"✅ {col}: 누락 없음")
        else:
            print(f"❌ 오류: 필수 필드 '{col}'가 데이터에 존재하지 않습니다.")

    # 3. 비정상적으로 짧은 혜택 내용 체크 (크롤링 오류 가능성)
    short_content = df[df['content'].str.len() < 20]
    if not short_content.empty:
        print(f"⚠️ 경고: 내용이 너무 짧은 데이터가 {len(short_content)}개 발견되었습니다. (예: {short_content['content'].iloc[0]})")

    # 4. 카드사별 데이터 분포 확인 (특정 카드사가 누락되었는지 확인용)
    print("\n🏢 카드사별 수집 현황 (상위 5개):")
    print(df['metadata.corp'].value_counts().head())
    
    print("-" * 50)

# 실행 (파일명에 맞춰 수정하세요)
check_json_quality("all_credit_benefits_rag.json")
check_json_quality("all_check_benefits_rag.json")

--- [all_credit_benefits_rag.json] 데이터 품질 체크 시작 ---
📊 총 데이터 개수: 5418개
✅ content: 누락 없음
✅ metadata.card_id: 누락 없음
✅ metadata.card_name: 누락 없음
✅ metadata.category: 누락 없음
⚠️ 경고: 내용이 너무 짧은 데이터가 2개 발견되었습니다. (예: [넥슨 현대카드] 선택형 혜택: )

🏢 카드사별 수집 현황 (상위 5개):
metadata.corp
KB국민카드    683
신한카드      649
우리카드      599
삼성카드      568
현대카드      540
Name: count, dtype: int64
--------------------------------------------------
--- [all_check_benefits_rag.json] 데이터 품질 체크 시작 ---
📊 총 데이터 개수: 2345개
✅ content: 누락 없음
✅ metadata.card_id: 누락 없음
✅ metadata.card_name: 누락 없음
✅ metadata.category: 누락 없음

🏢 카드사별 수집 현황 (상위 5개):
metadata.corp
KB국민카드     485
신한카드       334
우리카드       253
MG새마을금고    158
NH농협카드     154
Name: count, dtype: int64
--------------------------------------------------


# 카테고리 수집

In [4]:
import json

# 데이터 로드
with open('./data/all_credit_benefits_rag.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# 카테고리 종류 수집
all_categories = sorted(list(set(item['metadata']['category'] for item in raw_data)))

print(f"📊 현재 발견된 카테고리 총 {len(all_categories)}개:")
print(all_categories)

📊 현재 발견된 카테고리 총 127개:
['APP', 'BC TOP', 'CJ ONE', 'KT', 'LGU+', 'OK캐쉬백', 'OTT/영화/문화', 'PP', 'SKT', 'SPA브랜드', 'SSM', '간편결제', '게임', '경기관람', '고속버스', '골프', '공과금', '공과금/렌탈', '공연/전시', '공항', '공항라운지', '공항라운지/PP', '교육/육아', '교통', '국내외가맹점', '국민행복', '금융', '기차', '기타', '네이버페이', '대중교통', '대한항공', '대형마트', '도서', '동물병원', '드럭스토어', '디지털구독', '라운지키', '레저/스포츠', '렌탈 ', '렌터카', '리조트', '마트/편의점', '멤버십포인트', '면세점', '모든가맹점', '무실적', '무이자할부', '문화센터', '바우처', '배달앱', '백화점', '베이커리', '병원', '병원/약국', '보험', '보험사', '뷰티/피트니스', '비즈니스', '삼성페이', '생활', '선택형', '소셜커머스', '쇼핑', '수수료우대', '아시아나항공', '아울렛', '애완동물', '약국', '어린이집', '여행/숙박', '여행사', '연회비지원', '영화', '온라인 여행사', '온라인쇼핑', '유의사항', '은행사', '음원사이트', '인테리어', '일반음식점', '자동차', '자동차/하이패스', '저가항공', '저녁', '적립', '전통시장', '점심', '정비', '제주항공', '제휴/PLCC', '주유', '주유소', '지역', '진에어', '충전소', '카드사', '카카오페이', '카페', '카페/디저트', '캐시백', '택시', '테마파크', '통신', '패밀리레스토랑', '패스트푸드', '편의점', '푸드', '프리미엄', '프리미엄 서비스', '피트니스', '하이브리드', '하이패스', '학습지', '학원', '할인', '항공권', '항공마일리지', '해외', '해외이용', '해외직구', '해피포인트', '헤어', '혜택 프로모

In [5]:
import json

# 데이터 로드
with open('./data/all_check_benefits_rag.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# 카테고리 종류 수집
all_categories = sorted(list(set(item['metadata']['category'] for item in raw_data)))

print(f"📊 현재 발견된 카테고리 총 {len(all_categories)}개:")
print(all_categories)

📊 현재 발견된 카테고리 총 113개:
['APP', 'BC TOP', 'CJ ONE', 'KT', 'LGU+', 'OK캐쉬백', 'OTT/영화/문화', 'SSM', '간편결제', '게임', '경기관람', '고속버스', '골프', '공과금', '공과금/렌탈', '공연/전시', '공항라운지', '공항라운지/PP', '교육/육아', '교통', '국내외가맹점', '국민행복', '금융', '기차', '기타', '네이버페이', '대중교통', '대한항공', '대형마트', '도서', '드럭스토어', '디지털구독', '레저/스포츠', '렌터카', '마트/편의점', '멤버십포인트', '면세점', '모든가맹점', '무실적', '바우처', '배달앱', '백화점', '베이커리', '병원', '병원/약국', '보험', '보험사', '뷰티/피트니스', '비즈니스', '생활', '선택형', '소셜커머스', '쇼핑', '수수료우대', '아시아나항공', '아이스크림', '아이행복', '애완동물', '약국', '어린이집', '여행/숙박', '연회비지원', '영화', '온라인쇼핑', '유의사항', '유치원', '은행사', '음원사이트', '일반음식점', '자동차', '저가항공', '적립', '전통시장', '점심', '정비', '제주항공', '제휴/PLCC', '주유', '주유소', '증권사', '지역', '직장인', '진에어', '충전소', '카드사', '카카오페이', '카페', '카페/디저트', '캐시백', '택시', '테마파크', '통신', '패밀리레스토랑', '패스트푸드', '편의점', '푸드', '프리미엄', '프리미엄 서비스', '하이브리드', '하이패스', '학원', '할인', '항공권', '항공마일리지', '해외', '해외이용', '해외직구', '해피포인트', '헤어', '혜택 프로모션', '호텔', '홈쇼핑', '화장품']


# 카테고리 분류 main, sub

수집된 원본 데이터의 카테고리는 카드사마다 명칭이 달라 매우 파편화되어 있습니다. 이를 통합 매핑 사전을 통해 대분류 체계로 재구성하여 RAG 모델의 검색 효율성을 극대화합니다.


검색 고도화: 사용자가 "쇼핑"이라고 검색했을 때 '마트', '백화점', '온라인쇼핑' 등 유사 카테고리를 한 번에 조회할 수 있습니다.

필터링 성능 향상: 벡터 유사도 검색 전, 대분류(main_category)를 활용한 메타데이터 필터링을 적용하여 관련 없는 검색 결과를 사전에 차단합니다.

사용자 경험(UX): 추천 결과 제시 시 "귀하의 소비 패턴 중 [쇼핑/생활] 분야에 특화된 카드입니다"와 같은 구조화된 답변이 가능해집니다.

In [6]:
import json
import os

# 1. 신용/체크카드 전체 카테고리를 커버하는 통합 매핑 사전 (보강됨)
category_mapping = {
    # 쇼핑/생활 (마트, 편의점, 온라인, 드럭스토어 등)
    '쇼핑': '쇼핑/생활', '대형마트': '쇼핑/생활', '온라인쇼핑': '쇼핑/생활', '백화점': '쇼핑/생활', '소셜커머스': '쇼핑/생활', 
    '홈쇼핑': '쇼핑/생활', '아울렛': '쇼핑/생활', 'SPA브랜드': '쇼핑/생활', 'SSM': '쇼핑/생활', '전통시장': '쇼핑/생활', 
    '마트/편의점': '쇼핑/생활', '편의점': '쇼핑/생활', '드럭스토어': '쇼핑/생활', '화장품': '쇼핑/생활', '인테리어': '쇼핑/생활',
    '생활': '쇼핑/생활', '배달앱': '쇼핑/생활',
    
    # 푸드/카페 (음식점, 카페, 아이스크림 등)
    '카페': '푸드/카페', '카페/디저트': '푸드/카페', '베이커리': '푸드/카페', '일반음식점': '푸드/카페', 
    '패스트푸드': '푸드/카페', '패밀리레스토랑': '푸드/카페', '점심': '푸드/카페', '저녁': '푸드/카페', '푸드': '푸드/카페',
    '아이스크림': '푸드/카페',
    
    # 교통/자동차
    '주유': '교통/자동차', '주유소': '교통/자동차', '충전소': '교통/자동차', '택시': '교통/자동차', '하이패스': '교통/자동차', 
    '자동차': '교통/자동차', '자동차/하이패스': '교통/자동차', '정비': '교통/자동차', '교통': '교통/자동차', 
    '대중교통': '교통/자동차', '고속버스': '교통/자동차', '기차': '교통/자동차', '하이브리드': '교통/자동차', '렌터카': '교통/자동차',
    
    # 여행/프리미엄 (항공, 호텔, 라운지 등)
    '여행/숙박': '여행/프리미엄', '호텔': '여행/프리미엄', '리조트': '여행/프리미엄', '항공권': '여행/프리미엄', '대한항공': '여행/프리미엄', 
    '아시아나항공': '여행/프리미엄', '저가항공': '여행/프리미엄', '제주항공': '여행/프리미엄', '진에어': '여행/프리미엄', '여행사': '여행/프리미엄', 
    '온라인 여행사': '여행/프리미엄', '해외': '여행/프리미엄', '해외이용': '여행/프리미엄', '해외직구': '여행/프리미엄', '공항': '여행/프리미엄', 
    '항공마일리지': '여행/프리미엄', 'PP': '여행/프리미엄', '공항라운지': '여행/프리미엄', '공항라운지/PP': '여행/프리미엄', 
    '라운지키': '여행/프리미엄', '프리미엄': '여행/프리미엄', '프리미엄 서비스': '여행/프리미엄', '바우처': '여행/프리미엄', '비즈니스': '여행/프리미엄', '면세점': '여행/프리미엄',
    
    # 디지털/통신/구독
    '통신': '디지털/통신', 'SKT': '디지털/통신', 'KT': '디지털/통신', 'LGU+': '디지털/통신', '디지털구독': '디지털/통신', 
    'OTT/영화/문화': '디지털/통신', '음원사이트': '디지털/통신', '게임': '디지털/통신', '네이버페이': '디지털/통신', 
    '삼성페이': '디지털/통신', '카카오페이': '디지털/통신', '간편결제': '디지털/통신', 'APP': '디지털/통신',
    
    # 교육/의료/육아 (체크카드 전용 카테고리 추가)
    '학원': '교육/의료/육아', '학습지': '교육/의료/육아', '교육/육아': '교육/의료/육아', '어린이집': '교육/의료/육아', '도서': '교육/의료/육아', 
    '병원': '교육/의료/육아', '약국': '교육/의료/육아', '병원/약국': '교육/의료/육아', '문화센터': '교육/의료/육아', '동물병원': '교육/의료/육아', 
    '애완동물': '교육/의료/육아', '국민행복': '교육/의료/육아', '아이행복': '교육/의료/육아', '유치원': '교육/의료/육아',
    
    # 금융/납부 (공과금, 보험, 증권 등)
    '공과금': '금융/납부', '공과금/렌탈': '금융/납부', '보험': '금융/납부', '보험사': '금융/납부', '금융': '금융/납부', 
    '은행사': '금융/납부', '수수료우대': '금융/납부', '렌탈': '금융/납부', '카드사': '금융/납부', '증권사': '금융/납부',
    
    # 문화/레저/스포츠
    '영화': '문화/레저', '테마파크': '문화/레저', '레저/스포츠': '문화/레저', '골프': '문화/레저', '경기관람': '문화/레저', 
    '공연/전시': '문화/레저', '피트니스': '문화/레저', '뷰티/피트니스': '문화/레저', '헤어': '문화/레저',
    
    # 기본/전가맹점 (적립, 할인, 실적 상관없는 혜택 등)
    '모든가맹점': '기본/전가맹점', '국내외가맹점': '기본/전가맹점', '무실적': '기본/전가맹점', '무이자할부': '기본/전가맹점', 
    '할인': '기본/전가맹점', '적립': '기본/전가맹점', '캐시백': '기본/전가맹점', '연회비지원': '기본/전가맹점', '선택형': '기본/전가맹점', '직장인': '기본/전가맹점',
    
    # 포인트/제휴 (특정 제휴사 멤버십)
    '멤버십포인트': '포인트/제휴', '해피포인트': '포인트/전유', 'OK캐쉬백': '포인트/제휴', 'CJ ONE': '포인트/제휴', 'BC TOP': '포인트/제휴', '제휴/PLCC': '포인트/제휴'
}

def process_card_file(file_path, output_path):
    # 파일 존재 여부 확인
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return

    # 데이터 로드
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 데이터 변환 실행
    for item in data:
        # 1. 카테고리 추출 및 공백 제거
        old_cat = item['metadata'].get('category', '기타').strip()
        
        # 2. 매핑 테이블에서 찾고, 없으면 '기타/안내'로 통합
        new_cat = category_mapping.get(old_cat, '기타/안내')
        
        # 3. 데이터 업데이트 (main_category 필드 추가)
        item['metadata']['main_category'] = new_cat
        item['metadata']['sub_category'] = old_cat

    # 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    # 결과 요약
    final_cats = set(item['metadata']['main_category'] for item in data)
    print(f"✅ {os.path.basename(file_path)} 통합 완료!")
    print(f"📊 대분류 개수: {len(final_cats)}개 | 결과 파일: {output_path}")

# --- 실행부 ---
# 1. 신용카드 처리
process_card_file('./data/all_credit_benefits_rag.json', './data/all_credit_benefits_category_rag.json')

# 2. 체크카드 처리
process_card_file('./data/all_check_benefits_rag.json', './data/all_check_benefits_category_rag.json')

✅ all_credit_benefits_rag.json 통합 완료!
📊 대분류 개수: 12개 | 결과 파일: ./data/all_credit_benefits_category_rag.json
✅ all_check_benefits_rag.json 통합 완료!
📊 대분류 개수: 12개 | 결과 파일: ./data/all_check_benefits_category_rag.json


# 분류된 카테고리 확인

In [7]:
import json

# 데이터 로드
with open('./data/all_credit_benefits_category_rag.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# 카테고리 종류 수집
all_categories = sorted(list(set(item['metadata']['main_category'] for item in raw_data)))

print(f"📊 현재 발견된 카테고리 총 {len(all_categories)}개:")
print(all_categories)

📊 현재 발견된 카테고리 총 12개:
['교육/의료/육아', '교통/자동차', '금융/납부', '기본/전가맹점', '기타/안내', '디지털/통신', '문화/레저', '쇼핑/생활', '여행/프리미엄', '포인트/전유', '포인트/제휴', '푸드/카페']


In [8]:
import json

# 데이터 로드
with open('./data/all_check_benefits_category_rag.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# 카테고리 종류 수집
all_categories = sorted(list(set(item['metadata']['main_category'] for item in raw_data)))

print(f"📊 현재 발견된 카테고리 총 {len(all_categories)}개:")
print(all_categories)

📊 현재 발견된 카테고리 총 12개:
['교육/의료/육아', '교통/자동차', '금융/납부', '기본/전가맹점', '기타/안내', '디지털/통신', '문화/레저', '쇼핑/생활', '여행/프리미엄', '포인트/전유', '포인트/제휴', '푸드/카페']


# 노이즈 제거 및 데이터 통합

임베딩 성능 향상: "Powered by Froala"와 같은 무의미한 반복 문구를 제거하여, 모델이 '카드 혜택'이라는 핵심 정보에만 집중하게 합니다.

토큰 효율성: 불필요한 공백과 특수문자를 줄여 LLM API 사용 시 토큰 비용을 절감합니다.

가독성 및 문맥 연결: 파편화된 기호(*, -, /)를 자연스러운 문장 부호나 공백으로 치환하여 문맥적 유사도 계산을 돕습니다.

In [9]:
import json
import re
import os

def clean_noise(text):
    if not text:
        return ""
    
    # [1] 에디터 홍보 문구 제거 (Froala 관련 노이즈)
    text = re.sub(r'Powered by Froala Editor', '', text)
    
    # [2] 중복 헤더 제거 (예: [신한카드 Mr.Life] 공과금 혜택:)
    # 패턴: 대괄호 안의 내용 + 뒤에 오는 '혜택:' 문구 삭제
    text = re.sub(r'\[.*?\] .*? 혜택:', '', text)
    
    # [3] 불필요한 특수문자 정돈
    # '*' 제거, '-'를 공백으로, '/'를 쉼표로 변환하여 문맥 연결성 강화
    text = text.replace('*', '').replace('-', ' ').replace('/', ', ')
    
    # [4] 과도한 공백 및 줄바꿈 정규화 (임베딩 성능에 중요)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# 1. 처리할 파일 리스트
file_list = [
    './data/all_credit_benefits_category_rag.json',
    './data/all_check_benefits_category_rag.json'
]

combined_data = []

# 2. 파일별 로드 및 정제 실행
print("🧹 데이터 정제 및 통합을 시작합니다...")

for file_path in file_list:
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            current_data = json.load(f)
            
            # 파일명에 따라 신용/체크 타입 태깅 (메타데이터 보강)
            card_type = '신용' if 'credit' in file_path else '체크'
            
            for item in current_data:
                item['metadata']['card_type'] = card_type # 카드 타입 저장
                item['cleaned_content'] = clean_noise(item.get('content', ''))
                combined_data.append(item)
            
            print(f"✅ {card_type}카드 {len(current_data)}건 정제 완료")

# 3. 정제 결과 확인
print(f"\n📊 통합 결과: 총 {len(combined_data)}건")
print("\n🔍 [정제 샘플 확인]")
if combined_data:
    sample = combined_data[0]
    print(f"카드명: {sample['metadata']['card_name']} ({sample['metadata']['card_type']})")
    print(f"이전: {sample['content'][:100]}...")
    print(f"이후: {sample['cleaned_content'][:100]}...")

# 4. 통합 결과 저장
final_path = './data/cleaned_total_card_data.json'
with open(final_path, 'w', encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=4)

print(f"\n🚀 1단계 완료! 통합 파일 저장됨: {final_path}")

🧹 데이터 정제 및 통합을 시작합니다...
✅ 신용카드 5418건 정제 완료
✅ 체크카드 2345건 정제 완료

📊 통합 결과: 총 7763건

🔍 [정제 샘플 확인]
카드명: 신한카드 Mr.Life (신용)
이전: [신한카드 Mr.Life] 공과금 혜택: 월납요금(공과금) 10% 할인서비스 - 전기요금 /도시가스요금/ SKT, LG U+, KT 통신요금 할인 * 인터넷/집전화/이동통신/결합상...
이후: 월납요금(공과금) 10% 할인서비스 전기요금 , 도시가스요금, SKT, LG U+, KT 통신요금 할인 인터넷, 집전화, 이동통신, 결합상품 포함 일 1회 할인 적용 1회 이용 금...

🚀 1단계 완료! 통합 파일 저장됨: ./data/cleaned_total_card_data.json


# 카드혜택 정형화

In [1]:
import os
import json
import time
from dotenv import load_dotenv
from openai import OpenAI
from tqdm.auto import tqdm # 진행률 표시를 위한 라이브러리

# .env 파일 로드
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 1단계에서 저장한 통합 파일 로드
with open('./data/cleaned_total_card_data.json', 'r', encoding='utf-8') as f:
    total_data = json.load(f)

print(f"✅ 데이터 로드 완료: 총 {len(total_data)}건")

✅ 데이터 로드 완료: 총 7763건


### 🤖 LLM 기반 데이터 정형화: 구조적 임베딩 전략

#### 1차 데이터 정제
- content 필드에 담긴 카드 혜택 정보는 현재 하나의 긴 문장으로 이어져 있어, 이를 그대로 벡터 DB에 저장할 경우 임베딩 및 RAG 검색 성능이 저하되는 문제가 발생합니다. 
- 이를 해결하기 위해 GPT-4o-mini를 활용하여 데이터를 9개의 핵심 필드로 분해(Decomposition)하고 정형화합니다.

In [9]:
import json

# 통합 정제 데이터 로드
with open('./data/cleaned_total_card_data.json', 'r', encoding='utf-8') as f:
    total_data = json.load(f)

# 3등분 계산
total_count = len(total_data)
chunk_size = total_count // 3

# 데이터 분할
part1 = total_data[:chunk_size]
part2 = total_data[chunk_size:chunk_size*2]
part3 = total_data[chunk_size*2:]

# 각 파일 저장
with open('./share/part_1.json', 'w', encoding='utf-8') as f: json.dump(part1, f, ensure_ascii=False, indent=4)
with open('./share/part_2.json', 'w', encoding='utf-8') as f: json.dump(part2, f, ensure_ascii=False, indent=4)
with open('./share/part_3.json', 'w', encoding='utf-8') as f: json.dump(part3, f, ensure_ascii=False, indent=4)

print(f"✅ 분할 완료: Part1({len(part1)}), Part2({len(part2)}), Part3({len(part3)})")

✅ 분할 완료: Part1(2587), Part2(2587), Part3(2589)


In [1]:
import os
import json
import time
from dotenv import load_dotenv
from openai import OpenAI
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 💡 팀원별 수정 포인트 ---
PART_NUMBER = 1  # 본인이 맡은 번호(1, 2, 3)
MAX_WORKERS = 10 # 동시에 보낼 API 요청 수 (5~10 사이 권장)
# --------------------------

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

INPUT_PATH = f'./data/part_{PART_NUMBER}.json'
OUTPUT_PATH = f'./data/final_structured_part_{PART_NUMBER}.json'

# 데이터 로드
with open(INPUT_PATH, 'r', encoding='utf-8') as f:
    total_data = json.load(f)

# 기존 결과 로드 (이어서 하기)
if os.path.exists(OUTPUT_PATH):
    with open(OUTPUT_PATH, 'r', encoding='utf-8') as f:
        final_results = json.load(f)
else:
    final_results = []

# 이미 처리된 card_id 추출 (중복 방지)
processed_ids = {item['metadata']['card_id'] for item in final_results if 'metadata' in item}

# 아직 처리하지 않은 데이터만 필터링
to_process = [item for item in total_data if item['metadata']['card_id'] not in processed_ids]

def process_item(item):
    """실제 API 호출 및 데이터 가공을 담당하는 단위 함수"""
    card_name = item['metadata']['card_name']
    content = item['cleaned_content']
    
    prompt = f"""
    카드 혜택 원문에서 정보를 추출하여 반드시 아래 JSON 형식을 반환하세요.
    [원문]: {content}
    [카드명]: {card_name}
    [반환 JSON 구조]:
    {{
        "summary": "한 문장 요약",
        "merchants": ["주요 가맹점 리스트"],
        "condition": "전월 실적 조건 요약",
        "benefit_detail": "할인/적립 수치 상세",
        "time_limit": "시간/요일 제약 (없으면 null)",
        "benefit_exclusions": ["혜택을 받을 수 없는 항목/업종"],
        "performance_exclusions": ["전월 실적 계산에서 제외되는 항목"],
        "grace_period": "신규 발급 시 실적 유예 정보 (없으면 null)",
        "payment_conditions": "특정 페이 결제, 오프라인 한정 등 결제 방식 제약",
        "additional_info": "위 항목에 해당하지 않는 고객센터, 카드소재, 고지정책 등"
    }}
    [규칙]: 용어는 '전월 실적'으로 통일하고, 정보가 없으면 '정보 없음' 또는 null을 사용하세요.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            response_format={ "type": "json_object" },
            temperature=0
        )
        res_json = json.loads(response.choices[0].message.content)
        
        # 가공된 데이터 합치기
        item['ai_structured'] = res_json
        item['embedding_input'] = (
            f"[{item['metadata']['corp']}] {item['metadata']['card_name']} | "
            f"{res_json['summary']} | 가맹점: {', '.join(res_json['merchants'])} | "
            f"조건: {res_json['condition']}"
        )
        return item
    except Exception as e:
        # 에러 발생 시 로그를 남기고 None 반환
        print(f"❌ Error in {card_name}: {e}")
        return None

# --- 병렬 실행 부분 ---
print(f"🚀 Part {PART_NUMBER} 시작 (대상: {len(to_process)}건 / 기완료: {len(processed_ids)}건)")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # 작업을 스케줄링
    futures = [executor.submit(process_item, item) for item in to_process]
    
    # 완료된 순서대로 수집 (tqdm으로 진행 상황 표시)
    for i, future in enumerate(tqdm(as_completed(futures), total=len(futures), desc=f"Part {PART_NUMBER}")):
        result = future.result()
        if result:
            final_results.append(result)
        
        # 50건마다 중간 저장 (병렬 처리이므로 조금 더 자주 저장하는 것이 안전함)
        if (i + 1) % 50 == 0:
            with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
                json.dump(final_results, f, ensure_ascii=False, indent=4)

# 최종 저장
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, ensure_ascii=False, indent=4)

print(f"✨ Part {PART_NUMBER} 모든 작업 완료! 저장됨: {OUTPUT_PATH}")

🚀 Part 1 시작 (대상: 2182건 / 기완료: 79건)


Part 1:   0%|          | 0/2182 [00:00<?, ?it/s]

✨ Part 1 모든 작업 완료! 저장됨: ./data/final_structured_part_1.json


In [2]:
import json

combined_data = []
for i in range(1, 4):
    file_path = f'./data/final_structured_part_{i}.json'
    with open(file_path, 'r', encoding='utf-8') as f:
        combined_data.extend(json.load(f))

# 최종 저장
FINAL_OUTPUT = './data/final_structured_card_total.json'
with open(FINAL_OUTPUT, 'w', encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=4)

print(f"🚀 병합 완료! 총 {len(combined_data)}건의 데이터셋이 완성되었습니다.")

🚀 병합 완료! 총 7757건의 데이터셋이 완성되었습니다.


In [ ]:
'''
    {
        "content": "[삼성 iD CARE 카드] 정비 혜택: 엔진오일(오일필터·에어클리너 포함) 교환 시 15,000원 현장할인(연 1회) 차량 안전점검 무료(연 1회) 타이어 펑크 수리 무료(연 1회, 1개 기준) 업종 할인 대상 타이어 위치 교환 무료(연 1회, 휠밸런스 2개 포함) 업종 할인 대상 차량정비 스피드메이트 - 연 기준 : 1.1~12.31 - 전월 이용금액 30만원 이상 시 제공 - 이용 전 삼성카드 차량 점검 서비스 이용 의사를 스피드메이트에 전달한 경우에 한함 - 엔진오일은 ZIC A 기준이며, 다른 고급엔진오일로 교환 시 적용이 제한될 수 있음 - 수입차, 1.4톤 이상 트럭 제외 - 할인점 내 입점매장 제외",
        "metadata": {
            "card_id": 2875,
            "card_name": "삼성 iD CARE 카드",
            "corp": "삼성카드",
            "category": "정비",
            "min_performance": 300000,
            "annual_fee": "국내전용 [20,000]원 / 해외겸용 [20,000]원",
            "main_category": "교통/자동차",
            "sub_category": "정비",
            "card_type": "신용"
        },
        "cleaned_content": "엔진오일(오일필터·에어클리너 포함) 교환 시 15,000원 현장할인(연 1회) 차량 안전점검 무료(연 1회) 타이어 펑크 수리 무료(연 1회, 1개 기준) 업종 할인 대상 타이어 위치 교환 무료(연 1회, 휠밸런스 2개 포함) 업종 할인 대상 차량정비 스피드메이트 연 기준 : 1.1~12.31 전월 이용금액 30만원 이상 시 제공 이용 전 삼성카드 차량 점검 서비스 이용 의사를 스피드메이트에 전달한 경우에 한함 엔진오일은 ZIC A 기준이며, 다른 고급엔진오일로 교환 시 적용이 제한될 수 있음 수입차, 1.4톤 이상 트럭 제외 할인점 내 입점매장 제외",
        "ai_structured": {
            "summary": "삼성 iD CARE 카드로 엔진오일 교환 시 할인 및 차량 안전점검 등의 혜택을 제공합니다.",
            "merchants": [
                "스피드메이트"
            ],
            "condition": "전월 이용금액 30만원 이상 시 제공",
            "benefit_detail": "엔진오일 교환 시 15,000원 현장할인(연 1회), 차량 안전점검 무료(연 1회), 타이어 펑크 수리 무료(연 1회, 1개 기준), 타이어 위치 교환 무료(연 1회, 휠밸런스 2개 포함)",
            "time_limit": "연 기준 : 1.1~12.31",
            "benefit_exclusions": [
                "수입차",
                "1.4톤 이상 트럭",
                "할인점 내 입점매장"
            ],
            "performance_exclusions": [
                "정보 없음"
            ],
            "grace_period": "정보 없음",
            "payment_conditions": "이용 전 삼성카드 차량 점검 서비스 이용 의사를 스피드메이트에 전달한 경우에 한함",
            "additional_info": "엔진오일은 ZIC A 기준이며, 다른 고급엔진오일로 교환 시 적용이 제한될 수 있음"
        },
        "embedding_input": "[삼성카드] 삼성 iD CARE 카드 | 삼성 iD CARE 카드로 엔진오일 교환 시 할인 및 차량 안전점검 등의 혜택을 제공합니다. | 가맹점: 스피드메이트 | 조건: 전월 이용금액 30만원 이상 시 제공"
    },
'''

#### 2차 데이터 정제

1. 목적: 1차 정제 결과물 중 비정형 텍스트로 남은 additional_info 내의 핵심 제약 조건을 구조화하여 RAG(검색) 엔진의 응답 정확도를 극대화함.
2. 대상: 전체 7,757건 중 유의미한 추가 정보가 포함된 5,711건.
3. 필요성

- 비정형 데이터의 한계: additional_info 내에 섞여 있는 결제 수단 제약, 실적 유예 기간 등의 정보는 단순 텍스트로 존재할 때 LLM이 오인식할 가능성이 있음.

- 스키마 고도화: 해당 정보를 payment_conditions, grace_period 등 별도 필드로 분리하여 검색 필터링 및 임베딩 품질을 개선함.

In [2]:
import json

# 확인하고 싶은 파일 경로 (pilot 또는 part_n)
FILE_PATH = './data/final_structured_card_total.json' 

with open(FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 유의미한 정보가 있는 건수 계산
valid_count = sum(
    1 for item in data 
    if item.get('ai_structured', {}).get('additional_info') 
    not in [None, "정보 없음", "없음", "null", "None", ""]
)

print(f"✅ 전체 데이터: {len(data)}건")
print(f"📝 유의미한 추가 정보가 있는 데이터: {valid_count}건")

✅ 전체 데이터: 7757건
📝 유의미한 추가 정보가 있는 데이터: 5711건


In [1]:
import json
import pandas as pd
from collections import Counter
import re

# 1. 데이터 로드
FILE_PATH = './data/final_structured_card_total.json' 
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    merged_data = json.load(f)

# 2. 안전한 데이터 추출 및 클렌징 함수
def clean_text(text):
    if not text: return ""
    # 불필요한 특수문자 제거 및 공백 정규화
    text = re.sub(r'[^\w\s]', ' ', str(text))
    return text.strip()

def extract_additional_info(item):
    info = item.get('ai_structured', {}).get('additional_info')
    
    if info is None: return ""
    
    # 딕셔너리나 리스트인 경우 문자열로 합침
    if isinstance(info, dict):
        return " ".join([str(v) for v in info.values()])
    if isinstance(info, list):
        return " ".join([str(i) for i in info])
        
    return str(info)

# 3. 분석 대상 텍스트 수집 (5,711건 추출)
valid_texts = []
for item in merged_data:
    info_text = extract_additional_info(item)
    # 유의미한 내용만 필터링
    if info_text and info_text not in ["정보 없음", "없음", "null", "None"]:
        valid_texts.append(clean_text(info_text))

# 4. 빈도 분석 (1단어 & 2단어 조합)
single_words = []
double_words = []

for text in valid_texts:
    words = text.split()
    single_words.extend(words)
    # 2단어 조합 (Bigram): "실적" + "제외" -> "실적 제외"
    for i in range(len(words)-1):
        double_words.append(f"{words[i]} {words[i+1]}")

# 5. 결과 시각화 준비
single_res = Counter(single_words).most_common(50)
double_res = Counter(double_words).most_common(50)

print(f"✅ 총 {len(valid_texts)}건의 유의미한 추가 정보를 분석했습니다.\n")

print("📌 [TOP 30 핵심 단어 빈도]")
display(pd.DataFrame(single_res[:30], columns=['단어', '빈도']))

print("\n📌 [TOP 30 핵심 키워드 조합 (Bigram)]")
display(pd.DataFrame(double_res[:30], columns=['키워드 조합', '빈도']))

✅ 총 5711건의 유의미한 추가 정보를 분석했습니다.

📌 [TOP 30 핵심 단어 빈도]


,단어,빈도
0,시,1488
1,및,1325
2,가능,1065
3,홈페이지,846
4,카드,717
5,확인,714
6,서비스,690
7,변경,660
8,제공,637
9,이용,629



📌 [TOP 30 핵심 키워드 조합 (Bigram)]


,키워드 조합,빈도
0,변경 시,437
1,확인 가능,431
2,자세한 내용은,368
3,부가서비스 변경,348
4,홈페이지 www,315
5,이용 시,236
6,시 홈페이지에,179
7,수 있음,152
8,개별 고지,152
9,홈페이지에 게시하고,145


In [3]:
import json
import os

# 1. 1차 정제 완료된 통합 데이터 로드
INPUT_PATH = './data/final_structured_card_total.json'
with open(INPUT_PATH, 'r', encoding='utf-8') as f:
    total_data = json.load(f)

# 2. 2차 정제 대상(5,711건) 필터링
# additional_info에 실제 내용이 있는 데이터만 골라냅니다.
to_process = [
    item for item in total_data 
    if item.get('ai_structured', {}).get('additional_info') not in [None, "정보 없음", "없음", "null", "None", ""]
]

# 3. 데이터 3등분
n = len(to_process)
part_size = n // 3

print(f"📊 전체 {len(total_data)}건 중 2차 정제 대상: {n}건")

for i in range(3):
    start = i * part_size
    end = (i + 1) * part_size if i < 2 else n
    part_data = to_process[start:end]
    
    output_file = f'./data/refine_part_{i+1}.json'
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(part_data, f, ensure_ascii=False, indent=4)
    print(f"✅ {output_file} 생성 완료: {len(part_data)}건")

📊 전체 7757건 중 2차 정제 대상: 5711건
✅ ./data/refine_part_1.json 생성 완료: 1903건
✅ ./data/refine_part_2.json 생성 완료: 1903건
✅ ./data/refine_part_3.json 생성 완료: 1905건


In [4]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 💡 수정 포인트 ---
PART_NUM = 1  # 본인 번호 입력 (1, 2, 3)
MAX_WORKERS = 10 
# --------------------

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

INPUT_PATH = f'./data/refine_part_{PART_NUM}.json'
OUTPUT_PATH = f'./data/refined_result_part_{PART_NUM}.json'

with open(INPUT_PATH, 'r', encoding='utf-8') as f:
    part_data = json.load(f)

def get_microscopic_refinement(item):
    card_name = item['metadata']['card_name']
    target_info = item['ai_structured'].get('additional_info', '')
    
    prompt = f"""
    아래 [유의사항]에서 핵심 제약 조건을 추출하여 반드시 JSON으로 반환하세요.
    [카드명]: {card_name}
    [유의사항]: {target_info}

    [반환 JSON 구조]:
    {{
        "payment_conditions": "특정 페이 결제 필수 조건이나 온/오프라인 제약 (없으면 null)",
        "performance_exclusions_addon": ["추가 실적 제외 항목들 (없으면 빈 리스트)"],
        "grace_period_detail": "신규 발급 실적 유예 조건 (없으면 null)",
        "limit_detail": "월간 통합 할인/적립 한도 정보 (없으면 null)",
        "remains": "나머지 단순 안내 문구"
    }}
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            response_format={ "type": "json_object" },
            temperature=0
        )
        res = json.loads(response.choices[0].message.content)
        
        # 데이터 업데이트 로직
        s = item['ai_structured']
        if res['payment_conditions']: s['payment_conditions'] = res['payment_conditions']
        if res['performance_exclusions_addon']:
            existing = set(s.get('performance_exclusions', []))
            s['performance_exclusions'] = list(existing.union(set(res['performance_exclusions_addon'])))
        if res['grace_period_detail']: s['grace_period'] = res['grace_period_detail']
        if res['limit_detail']: s['benefit_detail'] += f" (한도: {res['limit_detail']})"
        s['additional_info'] = res['remains'] if res['remains'] else "없음"
        
        return item
    except: return item

print(f"🚀 Part {PART_NUM} ({len(part_data)}건) 현미경 정제 시작...")
results = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(get_microscopic_refinement, item) for item in part_data]
    for future in tqdm(as_completed(futures), total=len(part_data), desc=f"Processing Part {PART_NUM}"):
        results.append(future.result())

with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)
print(f"✨ Part {PART_NUM} 완료! 저장됨: {OUTPUT_PATH}")

🚀 Part 1 (1903건) 현미경 정제 시작...


Processing Part 1:   0%|          | 0/1903 [00:00<?, ?it/s]

✨ Part 1 완료! 저장됨: ./data/refined_result_part_1.json


In [1]:
import json

# 1. 2차 정제 결과물 로드
refined_all = []
for i in range(1, 4):
    with open(f'./data/refined_result_part_{i}.json', 'r', encoding='utf-8') as f:
        refined_all.extend(json.load(f))

# 2. 원본 데이터 로드 (정제 안 했던 데이터 매칭용)
with open('./data/final_structured_card_total.json', 'r', encoding='utf-8') as f:
    original_data = json.load(f)

# 3. 정제 완료된 card_id 목록
processed_ids = {item['metadata']['card_id'] for item in refined_all}

# 4. 정제 대상이 아니었던 데이터 추출
unprocessed_data = [item for item in original_data if item['metadata']['card_id'] not in processed_ids]

# 5. 전체 결합
final_dataset = refined_all + unprocessed_data

# 6. 임베딩용 텍스트 최종 합성 (검색 품질 최적화)
for item in final_dataset:
    res = item['ai_structured']
    meta = item['metadata']
    item['embedding_input'] = (
        f"[{meta['corp']}] {meta['card_name']} | "
        f"요약: {res['summary']} | 가맹점: {', '.join(res['merchants'])} | "
        f"조건: {res['condition']} | 결제방식: {res.get('payment_conditions', '무관')} | "
        f"실적유예: {res.get('grace_period', '정보없음')}"
    )

with open('./data/FINAL_REFINED_CARD_DATA_V2.json', 'w', encoding='utf-8') as f:
    json.dump(final_dataset, f, ensure_ascii=False, indent=4)

print(f"🎊 모든 작업 완료! 총 {len(final_dataset)}건의 최종 데이터셋이 생성되었습니다.")

🎊 모든 작업 완료! 총 5734건의 최종 데이터셋이 생성되었습니다.


In [3]:
import json
import re

# 1. 파일 로드 (현재 작업 중인 최신 파일)
INPUT_PATH = './data/FINAL_REFINED_CARD_DATA_V2.json'
OUTPUT_PATH = './data/FINAL_REFINED_CARD_DATA_V3.json'

with open(INPUT_PATH, 'r', encoding='utf-8') as f:
    final_data = json.load(f)

# 2. 제거할 노이즈 패턴 (대소문자 무관, 앞뒤 공백 포함)
noise_pattern = re.compile(r'\s*Powered by Froala Editor\s*', re.IGNORECASE)

def deep_clean(text):
    if not isinstance(text, str):
        return text
    # 노이즈 제거 후 양끝 공백 정리
    return noise_pattern.sub(' ', text).strip()

# 3. 데이터 전수 조사 및 세척
cleaned_count = 0

for item in final_data:
    is_dirty = False
    
    # 1) 원본 content (가장 중요)
    if 'content' in item:
        before = item['content']
        item['content'] = deep_clean(item['content'])
        if before != item['content']: is_dirty = True
            
    # 2) 가공된 cleaned_content
    if 'cleaned_content' in item:
        item['cleaned_content'] = deep_clean(item['cleaned_content'])
        
    # 3) AI가 추출한 additional_info
    if 'ai_structured' in item and 'additional_info' in item['ai_structured']:
        item['ai_structured']['additional_info'] = deep_clean(item['ai_structured']['additional_info'])
        
    # 4) 최종 검색용 embedding_input
    if 'embedding_input' in item:
        item['embedding_input'] = deep_clean(item['embedding_input'])

    if is_dirty:
        cleaned_count += 1

# 4. 결과 저장
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump(final_data, f, ensure_ascii=False, indent=4)

print(f"✅ 딥 클리닝 완료!")
print(f"📝 'content' 필드에서 노이즈가 발견되어 세척된 카드: {cleaned_count}건")
print(f"💾 최종 클린 데이터셋: {OUTPUT_PATH}")

✅ 딥 클리닝 완료!
📝 'content' 필드에서 노이즈가 발견되어 세척된 카드: 1186건
💾 최종 클린 데이터셋: ./data/FINAL_REFINED_CARD_DATA_V3.json


In [4]:
import json
import hashlib

# 1. 파일 경로 설정
ORIGINAL_PATH = './data/final_structured_card_total.json'  # 7,757건 원본
REFINED_PATH = './data/FINAL_REFINED_CARD_DATA_V3.json'    # 2차 정제 결과 (5,734건)

with open(ORIGINAL_PATH, 'r', encoding='utf-8') as f:
    original_total = json.load(f)
with open(REFINED_PATH, 'r', encoding='utf-8') as f:
    refined_subset = json.load(f)

# 2. 고유 식별자 생성 함수 (ID + 혜택 원문 조합)
def get_unique_key(item):
    # 카드 ID와 content(원문)를 합쳐서 해시값을 만듭니다.
    content_str = str(item.get('content', ''))
    unique_str = f"{item['metadata']['card_id']}_{content_str}"
    return hashlib.md5(unique_str.encode()).hexdigest()

# 3. 2차 정제본을 딕셔너리로 맵핑 (업데이트용)
refined_map = {get_unique_key(item): item for item in refined_subset}

# 4. 원본 7,757건을 베이스로 업데이트 진행
final_dataset = []
update_count = 0

for item in original_total:
    u_key = get_unique_key(item)
    
    # 만약 이 행(청크)이 2차 정제된 결과가 있다면 그걸 사용
    if u_key in refined_map:
        final_dataset.append(refined_map[u_key])
        update_count += 1
    # 없다면 1차 정제본 그대로 유지
    else:
        final_dataset.append(item)

# 5. 임베딩 텍스트 최종 클리닝 및 동기화
for item in final_dataset:
    res = item['ai_structured']
    meta = item['metadata']
    
    # 텍스트 재합성 (2차 정제에서 뽑은 payment_conditions 등 반영)
    item['embedding_input'] = (
        f"[{meta['corp']}] {meta['card_name']} | "
        f"요약: {res.get('summary', '')} | 가맹점: {', '.join(res.get('merchants', []))} | "
        f"조건: {res.get('condition', '')} | "
        f"결제수단: {res.get('payment_conditions', '무관')} | "
        f"실적유예: {res.get('grace_period', '정보없음')}"
    ).replace("Powered by Froala Editor", "").strip()

# 6. 최종 저장
OUTPUT_PATH = './data/FINAL_MASTER_DATA_FIXED_7757.json'
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump(final_dataset, f, ensure_ascii=False, indent=4)

print(f"✅ 최종 병합 결과: {len(final_dataset)}건")
print(f"📝 업데이트된 청크 수: {update_count}건")

✅ 최종 병합 결과: 7757건
📝 업데이트된 청크 수: 4548건


In [5]:
import json
import pandas as pd
from collections import Counter

# 1. 파일 로드
FILE_PATH = './data/FINAL_MASTER_DATA_FIXED_7757.json'
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    master_data = json.load(f)

# 2. 데이터프레임 변환
df = pd.json_normalize(master_data)

print("===== 🏆 마스터 데이터 최종 점검 보고서 =====")
print(f"1. 전체 레코드 수: {len(df)}건")
if len(df) == 7757:
    print("   ✅ 목표 수치(7,757건)와 일치합니다.")
else:
    print(f"   ⚠️ 수치 불일치! (현재 {len(df)}건 / 목표 7,757건)")

# 3. 필수 필드 누락 여부
required_fields = ['metadata.card_id', 'ai_structured.summary', 'embedding_input']
missing_counts = {field: df[field].isnull().sum() for field in required_fields}
print("\n2. 필수 필드 누락 확인:")
for field, count in missing_counts.items():
    status = "OK" if count == 0 else f"⚠️ {count}건 누락"
    print(f"   - {field}: {status}")

# 4. 2차 정제 성과 측정 (확보율)
def is_valid_info(val):
    return val not in [None, "정보 없음", "없음", "null", "None", "", []]

payment_rate = df['ai_structured.payment_conditions'].apply(is_valid_info).mean() * 100
grace_rate = df['ai_structured.grace_period'].apply(is_valid_info).mean() * 100

print("\n3. 정밀 정제 필드 확보율:")
print(f"   - 결제 조건(payment_conditions): {payment_rate:.1f}%")
print(f"   - 실적 유예(grace_period): {grace_rate:.1f}%")

# 5. 노이즈 잔류 검사
noise_text = "Powered by Froala Editor"
noise_count = df['embedding_input'].str.contains(noise_text, case=False).sum()
print(f"\n4. 노이즈 제거 확인:")
print(f"   - '{noise_text}' 잔류 건수: {noise_count}건")

# 6. 임베딩 입력값 품질 (길이 분석)
df['embedding_len'] = df['embedding_input'].str.len()
print("\n5. 임베딩 입력값 길이 통계:")
print(df['embedding_len'].describe()[['mean', 'min', 'max']])

# 7. 실제 데이터 샘플 (하단 3건)
print("\n👀 최종 데이터 샘플 확인:")
pd.set_option('display.max_colwidth', 100)
display(df[['metadata.card_name', 'ai_structured.payment_conditions', 'ai_structured.grace_period', 'embedding_input']].tail(3))

===== 🏆 마스터 데이터 최종 점검 보고서 =====
1. 전체 레코드 수: 7757건
   ✅ 목표 수치(7,757건)와 일치합니다.

2. 필수 필드 누락 확인:
   - metadata.card_id: OK
   - ai_structured.summary: OK
   - embedding_input: OK

3. 정밀 정제 필드 확보율:
   - 결제 조건(payment_conditions): 45.7%
   - 실적 유예(grace_period): 31.7%

4. 노이즈 제거 확인:
   - 'Powered by Froala Editor' 잔류 건수: 0건

5. 임베딩 입력값 길이 통계:
mean    189.588372
min      91.000000
max     819.000000
Name: embedding_len, dtype: float64

👀 최종 데이터 샘플 확인:


,metadata.card_name,ai_structured.payment_conditions,ai_structured.grace_period,embedding_input
7754,해피포인트 우리체크카드,정보 없음,"발급일 기준 당일 포인트 적립 가능, 포인트 사용은 익일부터 가능","[우리카드] 해피포인트 우리체크카드 | 요약: 해피포인트 우리체크카드는 가맹점에서 포인트 적립 및 청구할인 혜택을 제공합니다. | 가맹점: 해피포인트카드 가맹점, 파리바게뜨..."
7755,해외에선 체크카드,정보 없음,해당카드를 생애 최초 발급받은 경우에 적용,[KB국민카드] 해외에선 체크카드 | 요약: 해외에서 체크카드 이용 시 다양한 혜택과 조건이 적용됩니다. | 가맹점: 해외 가맹점 | 조건: 전월 실적 기준으로 이용금액 합...
7756,해피포인트 우리체크카드,정보 없음,카드수령등록일로부터 다음달 말일까지는 이용실적 조건없이 서비스 제공,"[우리카드] 해피포인트 우리체크카드 | 요약: 해피포인트 우리체크카드는 다양한 부가서비스와 할인 혜택을 제공하며, 전월 실적에 따라 서비스가 적용됩니다. | 가맹점: 국내 ..."
